Dieses Jupyter-Notebook dient dazu, Sie in die Lage zu versetzen, selbst einfache Analysen per Topic-Modeling durchzuführen. Wie genau man ein Topic Model programmatisch umsetzt, ist nicht klausurrelevant, dies dient lediglich der Übung.

Das Notebook zeigt Ihnen die (bzw. eine mögliche) technische Implementierung mit Python und weist auf einige Dinge hin, die Sie beim Modellieren beachten sollten. Ich biete auch Zusatzinformationen zu weiteren Paketen, die bei späteren, komplizierteren Datenanalysen nützlich sein können (```pandas``` u.a.). 

Sie müssen nicht alles in diesem Notebook lesen. Alle Absätze, die *kursiv* gedruckt sind, beschreiben vorbereitende Algorithmen, Schleifen usw., die nicht für das Topic Modeling selbst unbedingt benötigt werden, en detail. Sie können den Code auch einfach ausführen und sich diese Details nicht durchlesen. Schaden kann es allerdings auch nicht... 

**Bedienungsanleitung**: Lesen Sie sich erst einmal das Notebook durch, insbesondere auch die Zusammenfassung. Versuchen Sie dann, zu experimentieren (die Aufgaben geben Ihnen dabei Vorschläge). Die abschließende Aufgabe ist für eigene Experimente gedacht, in denen Sie mit selbst ausgewählten Daten arbeiten. Wenn Sie weitere Informationen zu den Funktionen brauchen, googlen Sie diese am besten zusammen mit dem Modulnamen.

**Wie funktioniert ein Jupyter-Notebook?**
- Interaktive Arbeit: statt, dass das "Programm" insgesamt durchläuft, läuft der Code Codezelle für Codezelle bei durch. Das ermöglicht ein flexibles Experimentieren, gerade im Bereich Datenanalyse!
- Codezellen kann man einzeln ausführen: Hineinklicken und Strg+Enter
- Wenn eine Zelle noch nicht ausgeführt wurde, dessen Output von einer späteren Zelle benötigt wird, wird die spätere Zelle nicht funktionieren
- Text kann in den Codezellen und Textzellen beliebig verändert werden (Speichern mit Strg+s)
- Änderungen am gesamten Notebook speichert man ebenfalls mit Strg+s
- Neue Zellen können Sie in der Leiste ganz oben mit dem "+"-Zeichen einfügen und dann in dem Dropdown-Menü entweder als Code- oder Textzelle (Markdown) kennzeichnen
- Wenn Sie eine Textzellen (Markdown) anlegen, erscheint diese erst in "hübsch", wenn Sie Strg+Enter drücken
- Schreiben Sie ruhig Notizen in eigenen Textzellen (oder meinen)!
- Kopieren Sie ruhig Code in eigene Codezellen und probieren Sie Dinge aus!

**Bei allen Änderungen, die Sie machen, nicht vergessen, mit Strg+S zu speichern!**

### Importe

Neben den sicherlich schon bekannten Modulen ```string```, ```re``` und ```nltk``` sind drei für die Verarbeitung von (Text)daten sehr wichtige Module zu nennen.

- **pandas** liefert flexible Datenstrukturen, die tabellenartig sind und die sich sql-artig manipulieren lassen. Dies ist ein für die Verarbeitung und spätere Modellierung komplexer Daten unerlässliches Modul und Industriestandard
- **numpy** erwei)tert die Standard-Python-Module um verbesserte Verarbeitungsmöglichkeiten für numerische Daten. Liefert auch verbesserte Listen (Series genannt), die etwa Grundlage für die Datenstrukturen in ```pandas``` sind

Für Topic Modeling im Speziellen
- **gensim** bietet Objekte und Funktionen für Topic-Modeling und nimmt Ihnen die komplizierteren Dinge ab
- **pyLDAvis** bietet einfache (und schöne) Visualisierungsmöglichkeiten der Ergebnisse von Topic-Modellen, die im Browser erscheinen (oder hier im Jupyter-Notebook)

In [ ]:
## Mpdule für die Interaktion mit dem Betriebssystem (Dateien einlesen)
import os

## Module für die Stringverarbeitung
import string # Funktionen für strings
import re # Regular Expressions
import nltk # Natural Language Toolkit

## Module für vereinfachte Datenverarbeitung
import pandas as pd # Flexible, abfragbare Datenstrukturen: DataFrames (tabellenartig) 
import numpy as np # Für die Arbeit mit Zahlen (inklusive Methoden für Verfahren der linearen Algebra u.a.)

## Module für Topic-Modelling (und anderes, textbezogenes)
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.models import LdaModel

## Visualisierung (von LDA Topic-Modellen)
import pyLDAvis
import pyLDAvis.gensim # speziell nötig für den Output eines gensim LDA-Topic-Modells

### Daten, Daten einlesen und praktische Datenformate

Wir benutzen einen Standard-Datensatz, den wir für ein interpretierbares Ergebnis nicht groß verändern müssen. Themen sind in der Literatur etwas nebulöser als z.B. in Forenthreads, weshalb sich letztere für Übungszwecke besser eignen. Es handelt sich um 20.000 Posts aus 20 Newsgroups (wir nehmen ein ca. 12.000 Posts großes Subset), welcher in den 1990ern gesammelt wurde und sehr oft für erste Tests von Algorithmen der Textklassifikation eingesetzt wird. Es ist Teil des Machine Learning Pakets ```sklearn``` und kann per ```import``` importiert und darauffolgend per Funktion heruntergeladen werden.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

```fetch_20newsgroups()``` holt die Daten. Das resultierende Objekt, das wir mit der Variable ```dataset``` assoziiert haben, hat mehrere Attribute, in denen die eigentlichen Daten liegen. Es gibt eigene Attribute unter anderem für die Dateinamen, die Textdaten selbst sowie das Label der Newsgroup, in der der Post entstanden ist.

Nun schreiben wir dies in einen ```pandas``` DataFrame. Diese kann man sich als einfache Tabellen vorstellen, die im Hintergrund aber zahlreiche praktische Abfragen und Funktionen erlaubt. Zum Beispiel kann man wie bei ```SQL``` filtern, einzelne Spalten auswählen oder Tabellen über entsprechende Key-Spalten zusammenführen. Python ohne ```pandas``` bietet kein vergleichbares Format, so dass sich das Modul in der Datenverarbeitung als unverzichtbar herausgestellt hat. Daten für den Zeitraum einer Analyse so aufzubewahren ist flexibel und übersichtlicher, als sehr viele lange Listen und andere Konstrukte zu definieren.

*Mit ```pd.DataFrame()``` generieren wir einen Dataframe. Darin eröffnen wir in ```{}``` ein Dictionary, worin die Schlüssel in Anführungszeichen später die SPaltennamen sein werden. Die Werte dieser Schlüssel können alles mögliche sein, in diesem Fall aber die betreffenden Attribute aus dem 20newsgroups-Datenobjekt. Diese Listen müssen natürlich gleichlang sein, es soll ja alles in dieselbe Tabelle.*

*Daraufhin können wir mit ```data_all["filenames"]``` sehr einfach auf die Spalte zugreifen. Wenn man nichts weiter dahinterschreibt, wird einfach die Spalte ausgegeben. In diesem Fall ändern wir aber die Einträge, die noch den kompletten Dateipfad zu den Texten enthalten, auf die reinen Postnamen (mittels regulärem Ausdruck). Die Logik von ```apply``` und ```lambda```-Ausdrücken zu erlkären würde hier zu weit gehen, aber sie sind sehr gut offiziell dokumentiert. ****Ändern Sie hier bitte heckelenme auf Ihren eigenen Usernamen.***

*Schlußendlich können wir uns den "Kopf" des Dataframes anzeigen lassen. Wenn man in ```head()``` eine Zahl einträgt, werden genauso viele Zeilen ausgegeben. Standardmäßig sind es fünf.*

In [ ]:
data_all = pd.DataFrame({
        "filenames" : dataset.filenames,
        "texts" : dataset.data,
        "newsgroup": dataset.target
        })
data_all['filenames'] = data_all['filenames'].apply(lambda x: re.sub('^.*\\/20news-bydate-train\\/', '', x))
data_all.head(10)

Das für uns wichtige, die Texte und die Dateinamen (also die Kennzahlen der Beiträge), finden wir in den Spalten ```filenames``` und ```texts```. Die zugehörige Chiffre der Newsgroup findet sich in ```newsgroup```. Einzelne Spalten können wir folgendermaßen ansprechen und dann wie eine übliche Listenindizierung in der Länge einschränken. Hier sprechen wir zunächst die Spalte ```filenames``` an und dann die ersten 10 Werte darin:

In [ ]:
data_all['filenames'][0:10]

Einzelne Zeilen kann man ansprechen, indem man ```.loc``` (location) wie unten benutzt und dann erst die Zeile (bzw. eine Range der Form ```0:10```) und dann nach einem Komma die Spalte angibt (als Zahlenindex oder über den Spaltennamen). Gibt man bei Spalte oder Zeile ein ```:``` ein, werden alle Spalten oder Zeilen ausgegeben (je nachdem, ob rechts oder links vom Komma).

In [ ]:
data_all.loc[0 , :]

Mit dem Attribut ```shape``` finden wir heraus, wieviele Zeilen und Spalten unser Dataframe hat.

In [ ]:
data_all.shape

Das sind nicht wenige Zeilen und daher Texte. Wir wollen für unsere Demonstrationszwecke Texte haben, die sich möglichst klar voneinander unterscheiden und außerdem nicht so viele, damit die Berechnung schnell geht. Dafür haben wir die Newsgroups mit aufgenommen, die ja intern thematisch gut beschränkt sind. Nun können wir unseren Dataframe filtern, indem wir nur die Zeilen behalten, die bei ```newsgroup``` ein entsprechendes Label haben. Wenn wir die Newsgroups "comp.sys.mac.hardware", "talk.politics.mideast" und "rec.sport.hockey" wollen, sind das die Labels 4, 17 und 10 (schauen Sie ruhig oben bei den zehn ausgegebenen Zeilen aus dem DataFrame).

*Der nachfolgende Befehl fragt in jeder Zeile ab, ob der Wert unter ```newsgroup``` in einer Liste ```[4, 17, 10]```, also einer Liste der gewünschten Labels enthalten sein könnte. Dazu indizieren wir einfach, wie wir es schon gelernt haben und nehmen als Index diese Abfrage. Das Resultat einer solchen Abfrage ist eine Liste mit ```True, False, True, True usw.```. Wenn man eine solche Liste dann für die Indizierung der Zeilen nutzt, werden nur die Zeilen mit ```True``` widergegeben.*

In [ ]:
data = data_all[ data_all["newsgroup"].isin([4, 17, 10]) ]
data.shape

### Was ist ein Thema?

Wenn es um die quantitative Konzeptualisierung (oder: Operationalisierung) von Themen geht, kann man unterschiedliche auf Zählungen basierte Annahmen haben. Ein Thema könnte man unter anderem  auf folgende Weisen definieren (keine Angst, wenn Sie einige Begriffe nicht kennen):

1. **Wörter pro Dokument zählen und häufigste pro Dokument anschauen. Dahingehend sehr ähnliche Dokumente haben dasselbe Thema, dass sich durch die häufigsten Wörter beschreiben lässt**
2. **Wörter pro Dokument zählen und per Algebra so daran herumrechnen, dass man die Variation der vielen Wörter durch wenige, "dahinterstehende" (latente) Variablen ausdrücken kann. Die Korrelationen dieser Themenvariablen mit bestimmten Wörtern kennzeichnen ihren Inhalt. Dann kann man sich die Verteilungen dieser Variablen über die Texte anschauen**

    - **Variante 1 oder 2 mit Ausschluss bestimmter Wortarten, Berechnungen der Worthäufigkeiten über Sequenzen in den Dokumenten statt den ganzen Dokumenten, Berechnungen anderer Kennzahlen als Häufigkeit als Grundlage u.v.m.**


3. **Wörter pro Dokument zählen. Ein Thema ist einerseits eine Häufigkseitsverteilung von Wörtern und Themen verteilen sich in einer Verteilung über jedes Dokument (prozentual kommt jedes Thema überall vor). Ein Algorithmus würfelt diese Verteilungen nun so aus, dass sie so gut es geht die in den Daten vorgefundenen Verteilungen widerspiegeln. Dann hat man zwei Arten von Listen: Die häufigsten Worte pro Thema und die prozentualen Anteile von Themen an Texten.***

    - **Variante 3 mit den bereits erwähnten Vorverarbeitungen**

Variante 1 könnte eine Clusteranalyse sein, wie sie in anderer Form auch in der Stilometrie zum Einsatz kommt. Variante zwei ist ein Verfahren namens Principal Components Analysis, welches auch in der Stilometrie zum Einsatz kommt. Unter anderem Namen, **Latent Semantic Analysis / Indexing**, wird es auch als eines der ersten und weitverbreitesten Topic-Modelle eingesetzt (im Natural Language Processing, nicht den DH). Es wurde als beliebtestes Verfahren von Variante 3 abgelöst, der **Latent Dirichlet Allocation**. Dieses Verfahren liefert auch ohne Vorverarbeitung gut interpretierbare Ergebnisse (wenn gut eingestellt). Tatsächlich wird aber auch hier oft viel im Vorfeld an den Texten gearbeitet.

Eine weitere Möglichkeit und zunehmend die ```LDA``` ablösend sind **Word Embeddings**. Diese Gruppe von Verfahren nutzt **Neural Networks**, um Themen zu lernen. Die resultierenden "Themen" sind noch sehr viel einleuchtender, als der Output älterer Verfahren, wobei das Ergebnis auch hier stark von der Datenbasis abhängt, die für das Training des Algorithmus genutzt wurde. 

Für dieses Notebook befassen wir uns erst einmal nur mit der ```LDA```.

### Workflow für die Umsetzung

```gensim``` erwartet einen ```Dictionary``` und einen Corpus mit Texten. Der Corpus ist nur eine Liste mit den Texten als Strings und der ```Dictionary``` ist eine Liste an Worten.

Angenommen, ```text_data``` ist eine Liste mit den tokenisierten Texten (die ihrerseits aus Listen von Wörtern bestehen):

- ```dictionary = corpora.Dictionary(text_data)```
- ```corpus = [dictionary.doc2bow(text) for text in text_data]```

```doc2bow()``` konvertiert in das Bag of Words Format: Die Struktur des Textes geht verloren und nur die Token werden später gezählt (nicht aber, wie sie zusammen im Text vorkommen). 

Oftmals muss man die Texte aber noch vorverarbeiten. Das sind ähnliche Schritte, wie Sie sie aus der maschinellen Sprachverarbeitung kennen: Whitespace entfernen, Stoppwörter entfernen, auf Kleinschreibung konvertieren, Stemming usw.. In ```gensim``` geht das, indem man erst eine Liste anlegt mit dem, was man möchte und diese dann über die Methode ```preprocess_string``` anwendet.

*Hier definieren wir als ```filters``` erst die Liste mit Dingen, die wir den Texten gern antun möchten. Per list comprehension nehmen wir dann jeden Text in unserem ```pandas```-Dataframe, also in dessen Spalte ```texts``` (angesprochen über ```data.texts```) und nutzen ```preprocess_string()``` zusammen mit ```filters```, um daraus tokenisierte Texte anzufertigen.*

*Mit ```corpora.Dictionary(preprocessed)``` bauen wir aus dieser neuen Liste dann ein "Wörterbuch", in dem alle Token verzeichnet sind. Dieses Wörterbuch hat dann eine Methode namens ```doc2bow()```, die wir in einer weiteren list comprehension nutzen können, um unseren finale Corpus zu bauen.*

In [ ]:
filters = [lambda x: x.lower(),
           lambda x: x.translate(str.maketrans('', '', string.punctuation)),
           strip_multiple_whitespaces,
           strip_numeric,
           remove_stopwords,
           stem_text]

preprocessed = [preprocess_string(text, filters) for text in data.texts]
dictionary = corpora.Dictionary(preprocessed)
corpus = [dictionary.doc2bow(prep_text) for prep_text in preprocessed]

Wir müssen also nun in irgendeiner Form in ein Tabellenformat, eine Matrix, transformieren. Diese Transformation ist im NLP oft die zu einer **Document-Term-Matrix**. Eine Document-Term-Matrix ist eine Matrix, die Dokumentnamen als Zeilen hat und Worte / Token als Spalten: **Dokumente x Worte**

Wenn man so zählt und in eine Tabelle einträgt, geht natürlich sämtliche Information über die Struktur des Textes verloren, weshalb man auch **Bag of Words**-Modell sagt (dies gilt auch für stilometrische Analysen).

Die Document-Term-Matrix ist die Grundlage (nach einiger Vorverarbeitung) für viele Verfahren im Natural Language Processing. Wichtig ist aber auch, was in den eigentlichen Zellen dieser Matrix steht. Das kann zum Beispiel die Häufigkeit sein, aber auch andere Zahlen wie die relative Häufigkeit (Anteil eines Worts am Dokument).

### Latent Dirichlet Allocation

Hat man alle Vorbereitungen getroffen, ist der weitere Code recht simpel. Die Methode ```LdaModel()``` hat viele Parameter, aber erst einmal interessieren uns nur drei: 

* ```corpus```: Hier müssen Sie Ihren angefertigten Corpus eintragen
* ```num_topics```: Hier legen Sie fest, wie viele Themen Sie herausbekommen möchten. Was hier das "Optimum" ist, müssen Sie durch Herumprobieren herausfinden
* ```id2word```: Hier wird das Wörterbuch, also der Dictionary, eingetragen

Es gibt zahlreiche weitere Einstellungsmöglichkeiten, die wir uns aber erst später anschauen.

(Die Fehlermeldungen können Sie ignorieren)

In [ ]:
# Latent Dirichlet Allocation

lda = LdaModel(
        corpus = corpus,
        num_topics = 20,
        id2word = dictionary
        )

## Visualisierung

Das Modul ```pyLDAvis``` ermöglicht übersichtliche, interaktive Visualisierungen von Latent Dirichlet Allocation - Outputs. Ohne weitere Umschweife lassen wir so eine Visualisierung einmal durchlaufen.

Dazu müssen Sie das Modell erst einmal vorbereiten. Die Methode ```pyLDAvis.gensim.prepare()``` ermöglicht dies. Angeben müssen Sie:

1. das Modellobjekt, 
2. den Corpus / die DTM und 
3. den zugehörigen Dictionary.

In [ ]:
viz = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)

Links sehen Sie die Ähnlichkeitsbeziehungen der Themen zueinander, wobei die Größe der Kreise den Anteil der Themen am Gesamtcorpus ausmacht. Sie können sie anklicken, dann erscheinen rechts die häufigsten Begriffe dieses Themas. Rot sind dabei die Häufigkeiten innerhalb des Themas und blau die Häufigkeiten im Gesamtcorpus. Wie Sie sehen, gibt es auch hier einiges an "Rauschen", wobei sich einige der Themen durchaus den ausgewählten Foren zuordnen lassen könnten. Andere aber erscheinen eher wie grammatische Restkategorien.

Sie können das Ergebnis für die Visualisierung abändern, indem Sie oben rechts die "relevance metric" anpassen. Dieses Spezifikum von ```pyLDAvis``` erlaubt es, die häufigsten Termini *insgesamt* etwas herunterzugewichten. Je niedriger der Wert, desto weniger tauchen global häufige Wörter in einem Thema auf. Findet man einen guten Wert, so erscheinen die für ein Thema spezifischen Wörter oben. Man kann es aber auch übertreiben.

#### Aufgabe

- Ändern Sie die Anzahl der Themen und schauen Sie, wie sich das Ergebnis ändert
- Notieren Sie sich, welche Themananzahlen besser funktioniert haben und wie sich das geäußert hat

### LDA mit mehr Parametern

Wie gesagt kann man noch einiges mehr einstellen. Diese helfen dabei, das Ergebnis zu verfeinern. Obwohl ```gensim``` ein Topic Modeling Paket ist, bietet es nicht alle statistischen Einstellungsmöglichkeiten. Die, die wir haben, sind aber auch schon gut und einige für Sie relevante sind mit ihren Standardwerten unten im Code zu sehen:

- **passes**: Wie oft soll der Algorithmus durch den Corpus gehen, um seine Einordnung zu verfeinern?
- **alpha**: Wie sollen die Themen verteilt sein? Alle gleichmäßig häufig oder nur wenige sehr häufig? Hier kann man detailreich für jedes der in ```num_topics``` festgelegten Themen Vorgaben per Liste machen. Relevant ist erst einmal aber nur die Einstellung ```auto```, mit der man nicht den vorgegebenen symmetrischen Alphawert nimmt, sondern diesen aus den Daten schätzen lässt (das hört sich objektiver an, ist es statistisch gesehen aber nicht wirklich).
- **eta**: Hier handelt es sich um die Verteilung der Wörter (prozentual) auf die Themen. Kann ebenfalls auf ```auto``` gestellt werden
- **iterations**: Der LDA-Algorithmus hat verschiedene Phasen, in denen er verschiedene statistische Optimierungsmöglichkeiten nutzt. Die Anzahl der Iterationen legt fest, wie oft der Algorithmus mit allen Dokumenten im Corpus einen Anlauf für eine dieser Optimierungsmöglichkeiten machen soll. Das hört sich so an wie **passes**, passiert aber tatsächlich innerhalb eines einzigen **pass**.
- **chunksize**: Wie viele Dokumente für jeden Trainingsdurchlauf eingesetzt werden sollen. Je mehr, desto langsamer, aber "genauer"

In [ ]:
lda = LdaModel(
    corpus = corpus,
    id2word = dictionary,
    num_topics = 20,
    passes = 10,
    chunksize = 2000,
    alpha = 'auto',
    eta = 'auto',
    iterations = 50
        )

### Visualisierung

In [ ]:
viz = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)

Das Ergebnis ist nun wesentlich interpretierbarer und vermutlich mit etwas Zeitaufwand über verschiedene Einstellungen verfeinerbar. Für Zwecke des Information Retrieval, z.B. Themenindizierung von wissenschaftlichen Artikeln für eine Suchfunktion, ist das schon ein Fortschritt. Ein Wissenschaftler müsste sich aber immer noch Fragen stellen, ob dieses oder ein anderes Ergebnis in Bezug auf ein theoretisches Themenkonstrukt gleich gut sind, oder ob es klar schlechtere Einteilungen gibt.

### Aufgabe

Spielen Sie mit den Einstellungen des LDA-Algorithmus und suchen Sie nach besseren Ergebnissen


# Abschließende Aufgabe

Nun erstellen Sie Ihr eigenes Topic Model, mit Daten aus dem Newsgroup-Corpus. Suchen Sie sich dafür aus folgender Liste heraus, welche Foren Sie interessieren. 

*Nachfolgender Code wählt zunächst die beiden Spalten ```filenames``` und ```newsgroup``` im Dataframe ```data_all``` aus (der ja alle Texte enthält). Die Methode ```drop_duplicates()``` mit der Spezifikation ```newsgroup``` sorgt dann dafür, dass basierend auf dieser Spalte keine Duplikate vorkommen. Das Ergebnis ist eine Liste aller Newsgroups und entsprechender Dateinamen, damit Sie sie inhaltlich zuordnen können.*

In [ ]:
data_all[['filenames', 'newsgroup']].drop_duplicates('newsgroup')

Wählen Sie drei Newsgroups aus, die Sie gern analysieren würden und von denen Sie denken, dass sie möglicherweise gut zu kontrastieren sind. Dann bauen Sie Ihren eigenen Datensatz, wie ich es weiter oben für meine drei Gruppen schon demonstriert hatte. Hier müssen Sie nichts weiter tun, als statt 4, 17 und 10 die IDs der gewünschten Gruppen einzutragen. Sie dürfen auch mehr als drei nehmen, aber dann kann es sein, dass die Berechnungen etwas länger dauern.

In [ ]:
data = data_all[ data_all["newsgroup"].isin([15, 17, 10]) ]
data = data.reset_index().drop("index", axis=1)
data['filenames'] = data['filenames'].apply(lambda x: re.sub('^.*\\/20news-bydate-train\\/', '', x))

In [ ]:
# Anschauen:

data.head(10)

### LDA

Jetzt die LDA. Hier können Sie mit den Einstellungen experimentieren (siehe Erklärungen in Teil 2). Sie müssen das allerdings nicht tun und können sich z.B. auch auf die Einstellungen bei der Erstellung der DTM und/oder die eventuelle, vorherige Normalisierung im Sinne NER, POS-Tagging und Co. verlassen.

Achten Sie darauf, dass Sie die richtige DTM (entweder ```X``` oder ```X_tfidf```) und den entsprechenden Dictionary wählen.

In [ ]:
filters = [lambda x: x.lower(),
           lambda x: x.translate(str.maketrans('', '', string.punctuation)),
           strip_multiple_whitespaces,
           strip_numeric,
           remove_stopwords,
           stem_text]

preprocessed = [preprocess_string(text, filters) for text in data.texts]
dictionary = corpora.Dictionary(preprocessed)
corpus = [dictionary.doc2bow(prep_text) for prep_text in preprocessed]

In [ ]:
lda = LdaModel(
    corpus = corpus,
    id2word = dictionary,
    num_topics = 20,
    passes = 10,
    chunksize = 2000,
    alpha = 'auto',
    eta = 'auto',
    iterations = 50
        )

### Visualisierung

In [ ]:
viz = pyLDAvis.gensim.prepare(lda, Sparse2Corpus(X.transpose()), dictionary_X)

In [ ]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)